In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
%cd S:\PPMI_RNA\Processed
data = pd.read_csv('All_Counts_PatAndHealthy_Disease.csv')
data['DIAGNOSIS'] = data['DIAGNOSIS'].replace({'PD': 1, 'Control': 0})
data.head(1)

S:\PPMI_RNA\Processed


,HudAlphaID,DIAGNOSIS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0001,1,15,232,15,0,0,0,0,0,...,0,1,16,1,0,2,19,0,0,0


In [4]:
X = data.drop(['HudAlphaID'], axis = 1)
y = data['DIAGNOSIS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   DIAGNOSIS  ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  \
0          1                 15                232                 15   
1          0                  2                199                 10   

   ENSG00000243485.5  ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   

   ENSG00000240361.2  ENSG00000186092.6  ...  ENSG00000273739.1  \
0                  0                  0  ...                  0   
1                  1                  0  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  1                 16                  1                  0   
1                  0                  2                  0                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00000278625.1  \
0        

In [5]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [6]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7085308056872038

In [7]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=127,
                                          random_state=RandomState(MT19937) at 0x240A3544340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x240A3544340, verbose=1)

In [8]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  44


In [9]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [10]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,DIAGNOSIS,1
14074,ENSG00000196159.11,1
17481,ENSG00000120129.5,1


### Top 50

In [11]:
Top50.to_csv ('DEGs_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('DEGs_Boruta_All.csv', index = False)

In [12]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_4932\3456753518.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000076864.19,ENSG00000057757.9,ENSG00000273136.6,ENSG00000281741.2,ENSG00000226067.6,ENSG00000143776.18,ENSG00000162722.8,ENSG00000185304.14,ENSG00000164047.4,...,ENSG00000268529.1,ENSG00000275719.1,ENSG00000130433.7,ENSG00000189068.10,ENSG00000105048.16,ENSG00000124256.14,ENSG00000092758.17,ENSG00000100079.6,ENSG00000189060.5,ENSG00000100116.16
0,1,1653,12244,2592,88,776,958,53364,113,526,...,7,74,42,96,181,2715,122,558,1149,278
1,0,27,7505,3645,58,464,513,15444,365,448,...,7,36,20,366,109,1999,175,969,1331,203


### Top 100

In [13]:
Top100 = selected_rf_features.iloc[:101, :]
Top100.to_csv ('DEGs_Boruta_Top100.csv', index = False)
Top100.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
14074,ENSG00000196159.11,1


In [14]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top100_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_4932\2083121888.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000076864.19,ENSG00000057757.9,ENSG00000187010.20,ENSG00000159023.21,ENSG00000222112.1,ENSG00000273136.6,ENSG00000281741.2,ENSG00000226067.6,...,ENSG00000105048.16,ENSG00000100985.7,ENSG00000124256.14,ENSG00000092758.17,ENSG00000254709.7,ENSG00000100079.6,ENSG00000189060.5,ENSG00000100116.16,ENSG00000238222.3,ENSG00000157514.16
0,1,359,1653,12244,9,147789,544,2592,88,776,...,181,664,2715,122,162,558,1149,278,26,7394
1,0,157,27,7505,174,55654,526,3645,58,464,...,109,581,1999,175,251,969,1331,203,5,8233


### Top 250

In [15]:
Top250 = selected_rf_features.iloc[:251, :]
Top250.to_csv ('DEGs_Boruta_Top250.csv', index = False)
Top250.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
14074,ENSG00000196159.11,1


In [16]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top250_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_4932\3768863930.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000117115.12,ENSG00000076864.19,ENSG00000007968.6,ENSG00000057757.9,ENSG00000187010.20,ENSG00000159023.21,ENSG00000134686.18,ENSG00000222112.1,...,ENSG00000189060.5,ENSG00000100116.16,ENSG00000047597.6,ENSG00000238222.3,ENSG00000102145.13,ENSG00000102349.17,ENSG00000184481.16,ENSG00000157514.16,ENSG00000130830.14,ENSG00000155962.12
0,1,359,1679,1653,4105,12244,9,147789,3026,544,...,1149,278,8170,26,1826,321,10949,7394,25655,492
1,0,157,1745,27,1484,7505,174,55654,2778,526,...,1331,203,3354,5,624,333,3016,8233,14195,477


### Top 500

In [17]:
Top500 = selected_rf_features.iloc[:501, :]
Top500.to_csv ('DEGs_Boruta_Top500.csv', index = False)
Top500.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
14074,ENSG00000196159.11,1


In [18]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top500_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_4932\3667433340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000162413.16,ENSG00000189337.16,ENSG00000117115.12,ENSG00000158825.5,ENSG00000076864.19,ENSG00000007968.6,ENSG00000057757.9,ENSG00000187010.20,ENSG00000158156.7,...,ENSG00000158578.20,ENSG00000102349.17,ENSG00000184481.16,ENSG00000198088.10,ENSG00000157514.16,ENSG00000076770.14,ENSG00000233093.5,ENSG00000130830.14,ENSG00000155962.12,ENSG00000278384.1
0,1,2852,359,1679,825,1653,4105,12244,9,947,...,157363,321,10949,77,7394,49525,111,25655,492,409
1,0,2776,157,1745,529,27,1484,7505,174,934,...,92908,333,3016,82,8233,35787,79,14195,477,124
